In [4]:
# -*- coding: utf-8 -*-
"""Example of using DeepSVDD for outlier detection
"""
# Author: Rafal Bodziony <bodziony.rafal@gmail.com>
# License: BSD 2 clause

from __future__ import division
from __future__ import print_function

import os
import sys
import numpy as np
import time

# temporary solution for relative imports in case pyod is not installed
# if pyod is installed, no need to use the following line
sys.path.append(
    os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))

from pyod.models.deep_svdd import DeepSVDD
from pyod.utils.data import generate_data
from pyod.utils.data import evaluate_print

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

def main():
    
    train_x = []
    train_y = []
    # 读取训练数据集
    for i in range(5):
        file = "./dataSet/data_batch_%d"%(i+1)
        dicts = unpickle(file)
        labels = dicts[b'labels']
        data = dicts[b'data'].reshape(-1, 3, 32, 32) # 3*32x32
        data = data.transpose(0,2,3,1)

        for j in range(len(labels)):
            img_data = data[j]
            label = labels[j]
            train_x.append(img_data)
            train_y.append(label)
    # 读取测试数据集
    test_x = []
    test_y = []
    for i in range(1):
        # file = "./data_batch_%d"%(i+1)
        file = "./dataSet/test_batch"
        dicts = unpickle(file)
        labels = dicts[b'labels']
        data = dicts[b'data'].reshape(-1, 3, 32, 32) # 3*32x32
        data = data.transpose(0,2,3,1)

        for j in range(len(labels)):
            img_data = data[j]
            label = labels[j]
            test_x.append(img_data)
            test_y.append(label)
    train_x = np.array(train_x)
    train_y = np.array(train_y)
    test_x = np.array(test_x)
    test_y = np.array(test_y)
    
    contamination = 0.1  # percentage of outliers
    use_ae = False # hyperparameter for use ae architecture instead of simple NN
    random_state = 10 # if C is set to None use random_state
    # Generate sample data
    # Generate sample data
    
    nomal_label = 0
    norm_train_x = train_x[train_y == nomal_label]
    norm_train_x = norm_train_x.flatten()
    norm_train_x = norm_train_x.reshape(-1, 3072)
    
    test_x = test_x.flatten()
    test_x = test_x.reshape(-1, 3072)

    
    true_y = test_y.copy()
    true_y[test_y == nomal_label] = 1
    true_y[test_y != nomal_label] = 0

    # train DeepSVDD detector (Without-AE)
    clf_name = 'DeepSVDD'
    clf = DeepSVDD(use_ae=use_ae, epochs=50, contamination=contamination,
                   random_state=random_state)
    clf.fit(norm_train_x)

    # get the prediction labels and outlier scores of the training data
    y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)
    y_train_scores = clf.decision_scores_  # raw outlier scores
    for i in range(10):
        start_time = time.time()
        y_test_pred = clf.predict(test_x)  # outlier labels (0 or 1)
        print("平均消耗的时间为", ((time.time()-start_time) / test_x.shape[0]*1000))

In [5]:
main()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 3072)]            0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                196672    
_________________________________________________________________
net_output (Dense)           (None, 32)                2080      
_________________________________________________________________
tf_op_layer_Sub_1 (TensorFlo [(None, 32)]              0         
_________________________________________________________________
tf_op_layer_Pow_1 (TensorFlo [(None, 32)]              0         
_________________________________________________________________
tf_op_layer_Sum_1 (TensorFlo [(None,)]                 0         
_________________________________________________________________
tf_op_layer_Mean_1 (TensorFl [()]                      0   